In [1]:
import numpy as np
import numpy.random as random
import scipy as sp
import pandas as pd
from pandas import Series,DataFrame

# 視覺化函式庫
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
%matplotlib inline

# 機器學習函式庫
import sklearn

# 顯示到小數點後第3位
%precision 3

'%.3f'

In [23]:
import requests, zipfile
import io

# 取得汽車價格資料
url = 'http://archive.ics.uci.edu/ml/machine-learning-databases/autos/imports-85.data'
res = requests.get(url).content

# 將取得的資料作為DataFrame物件讀取
auto = pd.read_csv(io.StringIO(res.decode('utf-8')),header=None)

# 在資料的行裡設定標籤
auto.columns = ['symboling','normalized-losses','make','fuel-type','aspiration','num-of-doors',
                'body-style','drive-wheels','engine-location','wheel-base','length','width','height',
                'curb-weight','engine-type','num-of-cylinders','engine-size','fuel-system','bore',
                'stroke','compression-ratio','horsepower','peak-rpm','city-mpg','highway-mpg','price']

In [4]:
print('汽車資料的形式：{}'.format(auto.shape))

汽車資料的形式：(205, 26)


In [5]:
auto.head()

,symboling,normalized-losses,make,fuel-type,aspiration,num-of-doors,body-style,drive-wheels,engine-location,wheel-base,...,engine-size,fuel-system,bore,stroke,compression-ratio,horsepower,peak-rpm,city-mpg,highway-mpg,price
0,3,?,alfa-romero,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,13495
1,3,?,alfa-romero,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,16500
2,1,?,alfa-romero,gas,std,two,hatchback,rwd,front,94.5,...,152,mpfi,2.68,3.47,9.0,154,5000,19,26,16500
3,2,164,audi,gas,std,four,sedan,fwd,front,99.8,...,109,mpfi,3.19,3.40,10.0,102,5500,24,30,13950
4,2,164,audi,gas,std,four,sedan,4wd,front,99.4,...,136,mpfi,3.19,3.40,8.0,115,5500,18,22,17450


#### 製作從horsepower、width、height這些解釋變數來預測price這個目標變數之模型

#### 8-2-2 資料的整理

In [6]:
# 計算各個行（欄位）裡有多少個「？」
auto = auto[['price','horsepower','width','height']]
auto.isin(['?']).sum()

price         4
horsepower    2
width         0
height        0
dtype: int64

In [7]:
# 將?取代為NaN，刪除有NaN的列
auto = auto.replace('?',np.nan).dropna()
print('汽車資料的形式:{}'.format(auto.shape))

汽車資料的形式:(199, 4)


In [13]:
print('資料型別的確認（型別轉換前）\n{}\n'.format(auto.dtypes))

資料型別的確認（型別轉換前）
price          object
horsepower     object
width         float64
height        float64
dtype: object



In [14]:
auto = auto.assign(price = pd.to_numeric(auto.price))
auto = auto.assign(horsepower = pd.to_numeric(auto.horsepower))
print('資料型別的確認（型別轉換前）\n{}\n'.format(auto.dtypes))

資料型別的確認（型別轉換前）
price           int64
horsepower      int64
width         float64
height        float64
dtype: object



In [15]:
# 確認各個變數的相關性
auto.corr()

,price,horsepower,width,height
price,1.000000,0.810533,0.753871,0.134990
horsepower,0.810533,1.000000,0.615315,-0.087407
width,0.753871,0.615315,1.000000,0.309223
height,0.134990,-0.087407,0.309223,1.000000


#### 8-2-3 模型建構與評估

In [16]:
# 為了資料分割（訓練資料與測試資料）的匯入
from sklearn.model_selection import train_test_split

# 為了多元線性迴歸模型建構的匯入
from sklearn.linear_model import LinearRegression

# 指定目標變數為price、其他為解釋變數
X = auto.drop('price', axis = 1)
y = auto['price']

# 分為訓練資料與測試資料
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .5, random_state = 0) # test_size = .5 為將資料分為一半、random_state將亂數固定

# 多元線性迴歸的初始化學習
model = LinearRegression()
model.fit(X_train,y_train)

# 顯示決定係數
print('決定係數（train）:{:.3f}'.format(model.score(X_train,y_train)))
print('決定係數（test）:{:.3f}'.format(model.score(X_test,y_test)))

# 顯示迴歸係數與截距
print('\n迴歸係數\n{}'.format(pd.Series(model.coef_,index = X.columns)))
print('截距：{:.3f}'.format(model.intercept_))

決定係數（train）:0.733
決定係數（test）:0.737

迴歸係數
horsepower      81.651078
width         1829.174506
height         229.510077
dtype: float64
截距：-128409.046


#### 練習問題8-1

In [26]:
auto = auto[['price','engine-size','width']]
auto = auto.replace('?',np.nan).dropna()
auto = auto.assign(price = pd.to_numeric(auto.price))

In [27]:
# 為了資料分割（訓練資料與測試資料）的匯入
from sklearn.model_selection import train_test_split

# 為了多元線性迴歸模型建構的匯入
from sklearn.linear_model import LinearRegression

# 指定目標變數為price、其他為解釋變數
X = auto[['width','engine-size']]
y = auto['price']

# 分為訓練資料與測試資料
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .5, random_state = 0) # test_size = .5 為將資料分為一半、random_state將亂數固定

# 多元線性迴歸的初始化學習
model = LinearRegression()
model.fit(X_train,y_train)

# 顯示決定係數
print('決定係數（train）:{:.3f}'.format(model.score(X_train,y_train)))
print('決定係數（test）:{:.3f}'.format(model.score(X_test,y_test)))

# 顯示迴歸係數與截距
print('\n迴歸係數\n{}'.format(pd.Series(model.coef_,index = X.columns)))
print('截距：{:.3f}'.format(model.intercept_))

決定係數（train）:0.783
決定係數（test）:0.778

迴歸係數
width          1261.735518
engine-size     109.526787
dtype: float64
截距：-84060.643
